In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions 
from pyspark.sql.functions import hour, split
from pyspark.sql.types import StructType,StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import *
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
sc = pyspark.SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/25 22:45:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark = pyspark.SQLContext.getOrCreate(sc)

/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [3]:
from pyspark.sql.functions import col,max,count,avg

In [4]:
#Path to the dataset
dir_path = 'shared/midterm/drive_stats_2019_Q1/'

In [5]:
from os import listdir
total_files = [dir_path + f for f in listdir(dir_path)]
total_files.sort()

In [6]:
TabA = total_files[7:]
TabB = total_files[:7]

In [7]:
data_A = spark.read.csv(TabA, sep=',',inferSchema=True, header=True)
data_B = spark.read.csv(TabB, sep=',',inferSchema=True, header=True)

In [8]:
#Installing and importing Bloom Filter
!pip install bloom_filter2
from bloom_filter2 import BloomFilter

Defaulting to user installation because normal site-packages is not writeable


In [9]:
capacity = 1000
t = BloomFilter(capacity, error_rate=0.001)

In [18]:
dist_mods = data_B.select('model').distinct()
cnt = dist_mods.count()
for i in range(cnt):
    t.add(dist_mods.collect()[i]['model'])

In [22]:
bf = sc.broadcast(t)

In [23]:
final_ans = data_A.select('model').distinct()

In [24]:
final_ans.rdd.map(lambda x: [x[0],x[0] in bf.value]).toDF(['Model','Found']).filter(col('Found')=='true').select('Model').show()



+--------------------+
|               Model|
+--------------------+
|         ST4000DM000|
|       ST12000NM0007|
|         ST8000DM005|
|          ST320LT007|
| TOSHIBA MQ01ABF050M|
|        ST8000NM0055|
|Seagate BarraCuda...|
| TOSHIBA MG07ACA14TA|
|        WDC WD60EFRX|
|         ST8000DM002|
|         ST4000DM005|
|HGST HUS726040ALE610|
|     TOSHIBA HDWF180|
|HGST HMS5C4040ALE640|
|HGST HUH721010ALE600|
| TOSHIBA MD04ABA500V|
| TOSHIBA MD04ABA400V|
|       ST10000NM0086|
|      WDC WD2500AAJS|
|  TOSHIBA MQ01ABF050|
+--------------------+
only showing top 20 rows

